In [3]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
from ase import Atom, Atoms
from ase.data import chemical_symbols, covalent_radii, vdw_alvarez
from ase.io import read, write
from pymatgen.core import Element
from scipy import stats
from tqdm.auto import tqdm

from mlip_arena.models import MLIPEnum, REGISTRY

%matplotlib inline

In [7]:
for model in MLIPEnum:
    
    model_name = model.name
    
    if model_name != 'eqV2(OMat)':
        continue
    
    print(f"========== {model_name} ==========")

    calc = MLIPEnum[model_name].value()

    for symbol in tqdm(chemical_symbols[1:]):

        s = set([symbol])

        if "X" in s:
            continue

        try:
            atom = Atom(symbol)
            rmin = 0.9 * covalent_radii[atom.number]
            rvdw = vdw_alvarez.vdw_radii[atom.number] if atom.number < len(vdw_alvarez.vdw_radii) else np.nan
            rmax = 3.1 * rvdw if not np.isnan(rvdw) else 6
            rstep = 0.01

            a = 2 * rmax

            npts = int((rmax - rmin)/rstep)

            rs = np.linspace(rmin, rmax, npts)
            es = np.zeros_like(rs)

            da = symbol + symbol

            out_dir = Path(REGISTRY[model_name]["family"]) / str(da)
            os.makedirs(out_dir, exist_ok=True)

            skip = 0

            element = Element(symbol)

            try:
                m = element.valence[1]
                if element.valence == (0, 2):
                    m = 0
            except:
                m = 0


            r = rs[0]

            positions = [
                [a/2-r/2, a/2, a/2],
                [a/2+r/2, a/2, a/2],
            ]

            traj_fpath = out_dir / f"{model_name}.extxyz"

            if traj_fpath.exists():
                traj = read(traj_fpath, index=":")
                skip = len(traj)
                atoms = traj[-1]
            else:
                # Create the unit cell with two atoms
                atoms = Atoms(
                    da,
                    positions=positions,
                    # magmoms=magmoms,
                    cell=[a, a+0.001, a+0.002],
                    pbc=True
                )

            print(atoms)

            atoms.calc = calc

            for i, r in enumerate(tqdm(rs)):

                if i < skip:
                    continue

                positions = [
                    [a/2-r/2, a/2, a/2],
                    [a/2+r/2, a/2, a/2],
                ]

                # atoms.set_initial_magnetic_moments(magmoms)

                atoms.set_positions(positions)

                es[i] = atoms.get_potential_energy()

                write(traj_fpath, atoms, append="a")
        except Exception as e:
            print(e)


========== eqV2(OMat) ==========


  0%|          | 0/118 [00:00<?, ?it/s]

Atoms(symbols='H2', pbc=True, cell=[7.4399999999999995, 7.441, 7.441999999999999], calculator=SinglePointCalculator(...))


  0%|          | 0/344 [00:00<?, ?it/s]

Atoms(symbols='He2', pbc=True, cell=[8.866, 8.866999999999999, 8.868], calculator=SinglePointCalculator(...))


...

Atomic number exceeds that given in model config
Atoms(symbols='Ts2', pbc=True, cell=[12.0, 12.001, 12.002])


  0%|          | 0/582 [00:00<?, ?it/s]

Atomic number exceeds that given in model config
Atoms(symbols='Og2', pbc=True, cell=[12.0, 12.001, 12.002])


  0%|          | 0/582 [00:00<?, ?it/s]

Atomic number exceeds that given in model config


In [2]:


for model in MLIPEnum:
    
    model_name = model.name
    
    json_fpath = Path(REGISTRY[model_name]["family"]) / "homonuclear-diatomics.json"
    
    if json_fpath.exists():
        continue

    print(f"========== {model_name} ==========")
    
    df = pd.DataFrame(columns=[
        "name", 
        "method", 
        "R", "E", "F", "S^2", 
        "force-flip-times",
        "force-total-variation",
        "force-jump",
        "energy-diff-flip-times",
        "energy-grad-norm-max",
        "energy-jump",
        "energy-total-variation",
        "tortuosity",
        "conservation-deviation",
        "spearman-descending-force",
        "spearman-ascending-force",
        "spearman-repulsion-energy",
        "spearman-attraction-energy"
    ])
    

    for symbol in tqdm(chemical_symbols[1:]):

        da = symbol + symbol

        out_dir = Path(REGISTRY[model_name]["family"]) / da

        traj_fpath = out_dir / f"{model_name}.extxyz"


        if traj_fpath.exists():
            traj = read(traj_fpath, index=":")
        else:
            continue

        Rs, Es, Fs, S2s = [], [], [], []
        for atoms in traj:

            vec = atoms.positions[1] - atoms.positions[0]
            r = np.linalg.norm(vec)
            e = atoms.get_potential_energy()
            f = np.inner(vec/r, atoms.get_forces()[1])
            # s2 = np.mean(np.power(atoms.get_magnetic_moments(), 2))

            Rs.append(r)
            Es.append(e)
            Fs.append(f)
            # S2s.append(s2)

        rs = np.array(Rs)
        es = np.array(Es)
        fs = np.array(Fs)

        indices = np.argsort(rs)[::-1]
        rs = rs[indices]
        es = es[indices]
        eshift = es[0]
        es -= eshift
        fs = fs[indices]

        iminf = np.argmin(fs)
        imine = np.argmin(es)

        de_dr = np.gradient(es, rs)
        d2e_dr2 = np.gradient(de_dr, rs)

        rounded_fs = np.copy(fs)
        rounded_fs[np.abs(rounded_fs) < 1e-2] = 0
        fs_sign = np.sign(rounded_fs)
        mask = fs_sign != 0
        rounded_fs = rounded_fs[mask]
        fs_sign = fs_sign[mask]
        f_flip = np.diff(fs_sign) != 0
        
        fdiff = np.diff(fs)
        fdiff_sign = np.sign(fdiff)
        mask = fdiff_sign != 0
        fdiff = fdiff[mask]
        fdiff_sign = fdiff_sign[mask]
        fdiff_flip = np.diff(fdiff_sign) != 0
        fjump = np.abs(fdiff[:-1][fdiff_flip]).sum() + np.abs(fdiff[1:][fdiff_flip]).sum()
        

        ediff = np.diff(es)
        ediff[np.abs(ediff) < 1e-3] = 0
        ediff_sign = np.sign(ediff)
        mask = ediff_sign != 0
        ediff = ediff[mask]
        ediff_sign = ediff_sign[mask]
        ediff_flip = np.diff(ediff_sign) != 0
        ejump = np.abs(ediff[:-1][ediff_flip]).sum() + np.abs(ediff[1:][ediff_flip]).sum()
        
        
#         edged_es = np.convolve(es, [1, -2, 1], mode='valid')
#         # edged_es[np.abs(edged_es) < 0.1] = 0
#         prob = np.exp(-es[1:-1]) / np.sum(np.exp(-es[1:-1]))
#         edged_es *= prob
#         # edged_es /= np.abs(es[1:-1])
#         ejump = np.linalg.norm(edged_es)
#         ejump = np.abs(edged_es).sum() / 2.0
        
#         edged_fs = np.convolve(fs, [1, -2, 1], mode='valid')
#         # edged_fs[np.abs(edged_fs) < 0.1] = 0
#         edged_fs *= prob
#         fjump = np.linalg.norm(edged_fs)
        # fjump = np.abs(edged_fs).sum() / 2.0
        
#         fig, axes = plt.subplot_mosaic(
#             """
#             ac
#             bd
#             """,
#             constrained_layout=True
#         )
        

#         axes['a'].plot(rs, es)
#         axes['b'].plot(rs[1:-1], edged_es)
#         # axes['b'].plot(0.5*(rs[1:] + rs[:-1]), np.diff(es))
#         axes['b'].text(0.7, 0.7, f"{ejump:.3e}", transform=axes['b'].transAxes)
        
#         axes['c'].plot(rs, fs)
#         axes['d'].plot(rs[1:-1], edged_fs)
#         axes['d'].text(0.7, 0.7, f"{fjump:.3e}", transform=axes['d'].transAxes)
        

        conservation_deviation = np.mean(np.abs(fs + de_dr))
        
        etv = np.sum(np.abs(np.diff(es)))

        data = {
            "name": da,
            "method": model_name,
            "R": rs,
            "E": es + eshift,
            "F": fs,
            "S^2": S2s,
            "force-flip-times": np.sum(f_flip),
            "force-total-variation": np.sum(np.abs(np.diff(fs))),
            "force-jump": fjump,
            "energy-diff-flip-times": np.sum(ediff_flip),
            "energy-grad-norm-max": np.max(np.abs(de_dr)),
            "energy-jump": ejump,
            # "energy-grad-norm-mean": np.mean(de_dr_abs),
            "energy-total-variation": etv,
            "tortuosity": etv / (abs(es[0] - es.min()) + (es[-1] - es.min())),
            "conservation-deviation": conservation_deviation,
            "spearman-descending-force": stats.spearmanr(rs[iminf:], fs[iminf:]).statistic,
            "spearman-ascending-force": stats.spearmanr(rs[:iminf], fs[:iminf]).statistic,
            "spearman-repulsion-energy": stats.spearmanr(rs[imine:], es[imine:]).statistic,
            "spearman-attraction-energy": stats.spearmanr(rs[:imine], es[:imine]).statistic,
        }

        df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

    json_fpath = Path(REGISTRY[model_name]["family"]) / "homonuclear-diatomics.json"

    if json_fpath.exists():
        df0 = pd.read_json(json_fpath)
        df = pd.concat([df0, df], ignore_index=True)
        df.drop_duplicates(inplace=True, subset=["name", "method"], keep='last')

    df.to_json(json_fpath, orient="records")

========== MACE-MP(M) ==========


  0%|          | 0/118 [00:00<?, ?it/s]

/tmp/ipykernel_433641/3530606248.py:164: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
/tmp/ipykernel_433641/3530606248.py:156: RuntimeWarning: invalid value encountered in scalar divide
  "tortuosity": etv / (abs(es[0] - es.min()) + (es[-1] - es.min())),
/tmp/ipykernel_433641/3530606248.py:158: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  "spearman-descending-force": stats.spearmanr(rs[iminf:], fs[iminf:]).statistic,
/tmp/ipykernel_433641/3530606248.py:160: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  "spearman-repulsion-energy": stats.spearmanr(rs[imine:], es[imine:]).statistic

========== CHGNet ==========


  0%|          | 0/118 [00:00<?, ?it/s]

...

  0%|          | 0/118 [00:00<?, ?it/s]

/tmp/ipykernel_433641/3530606248.py:164: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)


========== ALIGNN ==========


  0%|          | 0/118 [00:00<?, ?it/s]

/tmp/ipykernel_433641/3530606248.py:164: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
/tmp/ipykernel_433641/3530606248.py:161: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  "spearman-attraction-energy": stats.spearmanr(rs[:imine], es[:imine]).statistic,


In [3]:
df

,name,method,R,E,F,S^2,force-flip-times,force-total-variation,energy-diff-flip-times,energy-grad-norm-max,energy-jump,energy-total-variation,conservation-deviation,spearman-descending-force,spearman-ascending-force,spearman-repulsion-energy,spearman-attraction-energy,tortuosity,force-jump
118,HH,ALIGNN,"[3.7199999999999998, 3.70996794, 3.69993586, 3...","[-1.2249419689178467, -1.2238645553588867, -1....","[1.91e-06, 0.00826454, 0.00533009, -0.00355052...",[],29,443.614255,30,40.656501,4.074248,13.676080,2.022960,-0.200684,-0.119952,-0.986572,0.844786,1.797762,458.210954
119,HeHe,ALIGNN,"[4.433, 4.4229736200000005, 4.41294724, 4.4029...","[2.4748411178588867, 2.4748411178588867, 2.474...","[0.0, -1e-08, 0.0, 0.0, 1e-08, 0.0, 0.0, 0.0, ...",[],44,1448.979436,43,160.175544,13.831072,32.544741,4.849522,-0.021494,-0.195001,-0.720218,0.609519,3.921846,1582.685197
120,LiLi,ALIGNN,"[6.572000000000001, 6.561981520000001, 6.55196...","[-0.21738338470458984, -0.21738338470458984, -...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],22,1127.749591,20,281.659136,2.067203,20.053466,2.469147,-0.714286,-0.444127,-0.991010,-0.654715,1.386696,1721.766498
121,BeBe,ALIGNN,"[6.138000000000001, 6.12797338, 6.117946759999...","[2.665587902069092, 2.665587902069092, 2.66558...","[0.0, 0.0, 0.0, 0.0, 1e-08, 0.0, 0.0, 0.0, 0.0...",[],25,1645.960357,21,145.040462,9.422094,51.647955,5.076523,0.544090,-0.158055,-0.989962,0.342476,1.715953,1775.001526
122,BB,ALIGNN,"[5.921000000000001, 5.91097088, 5.900941739999...","[0.6220548152923584, 0.6220548152923584, 0.622...","[0.0, 0.0, 1e-08, 0.0, 0.0, 1e-08, -1e-08, -1e...",[],35,1811.413732,36,131.340791,13.088368,52.691856,5.551787,0.052632,-0.171460,-0.985024,0.757880,1.617372,1799.183469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,FlFl,ALIGNN,"[6.0, 5.989982779999999, 5.979965579999999, 5....","[-10.127323150634766, -10.127323150634766, -10...","[0.0, 0.0, -9.5e-07, 0.0, 0.0, 0.0, 0.0, 9.5e-...",[],39,3329.770831,44,181.338142,12.292844,84.239434,8.151059,-0.249255,-0.316556,-0.946770,0.159576,2.694147,3926.252725
232,McMc,ALIGNN,"[6.0, 5.989982779999999, 5.979965579999999, 5....","[-10.127323150634766, -10.127323150634766, -10...","[0.0, 0.0, -9.5e-07, 0.0, 0.0, 0.0, 0.0, 9.5e-...",[],39,3329.770831,44,181.338142,12.292844,84.239434,8.151059,-0.249255,-0.316556,-0.946770,0.159576,2.694147,3926.252725
233,LvLv,ALIGNN,"[6.0, 5.989982779999999, 5.979965579999999, 5....","[-10.127323150634766, -10.127323150634766, -10...","[0.0, 0.0, -9.5e-07, 0.0, 0.0, 0.0, 0.0, 9.5e-...",[],39,3329.770831,44,181.338142,12.292844,84.239434,8.151059,-0.249255,-0.316556,-0.946770,0.159576,2.694147,3926.252725
234,TsTs,ALIGNN,"[6.0, 5.989982779999999, 5.979965579999999, 5....","[-10.127323150634766, -10.127323150634766, -10...","[0.0, 0.0, -9.5e-07, 0.0, 0.0, 0.0, 0.0, 9.5e-...",[],39,3329.770831,44,181.338142,12.292844,84.239434,8.151059,-0.249255,-0.316556,-0.946770,0.159576,2.694147,3926.252725
